# **Usando Python e dados abertos no Google Colab**

Bruno Gimenes Delphim

[*'Discente do curso de Biblioteconomia e Ciência da Informação - Campus USP-RP*](https://github.com/bgd94)

Neste notebook do Google Colab, eu vou mostrar como utilizar Python para acessar dados abertos do governo.

---

## Python

## catalogos-dados-brasil (**tableschema & datapackage**)
Conforme https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb

### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [ ]:
pip install datapackage tableschema-pandas plotly plotly_express

#### Imprimindo versões instaladas

In [ ]:
pip freeze | grep -e datapackage -e tableschema -e plotly

### Lendo o pacote de dados (**plotly**)

In [ ]:
import pandas as pd

# Para trabalhar com Frictionless Data – frictionlessdata.io
from tableschema import Storage
from datapackage import Package

# Para visualização
import plotly_express as px
import plotly as py, plotly.graph_objects as go


# Tentar gravar no Panda e imprimir "true" se for bem-sucedido
output = "true"
try:
    url = 'https://github.com/dadosgovbr/catalogos-dados-brasil/raw/master/datapackage.json'

    # Conectar ao armazenamento Pandas
    storage = Storage.connect('pandas')

    # Carregar o pacote de dados da URL e salvar no armazenamento
    package = Package(url)
    package.save(storage=storage)

    # Se não houver exceções, imprimir "true"


except Exception as e:
    # Se ocorrer uma exceção, imprimir "false" e a mensagem de erro
    output = "false"
    print(e)

output

storage.buckets

type(storage['catalogos'])

storage['solucao']

storage['catalogos'].head()

tipo_solucao = storage['catalogos'].groupby('Solução').count()['URL'].rename('quantidade')
tipo_solucao

px.bar(
    pd.DataFrame(tipo_solucao).reset_index(),
    x = 'Solução',
    y = 'quantidade',
    color = 'Solução',
    color_discrete_sequence = py.colors.qualitative.Set2
)

poder = storage['catalogos'].groupby('Poder').count()['URL'].rename('quantidade')
poder

go.Figure(
    data=go.Pie(
        labels=poder.index,
        values=poder.values,
        hole=.4
    )
).show()

esfera = storage['catalogos'].groupby('Esfera').count()['URL'].rename('quantidade')
esfera

go.Figure(
    data=go.Pie(
        labels=esfera.index,
        values=esfera.values,
        hole=.4
    )
).show()

uf = storage['catalogos'].groupby('UF').count()['URL'].rename('quantidade')
uf

px.bar(
    pd.DataFrame(uf).reset_index(),
    x = 'UF',
    y = 'quantidade',
    color = 'UF',
    color_discrete_sequence = py.colors.qualitative.Set3
)

# Filtrar os registros com campo "UF" igual a "SP"
filtered_data = storage['catalogos'][storage['catalogos']['UF'] == 'SP']

# Exibir os registros filtrados
filtered_data['URL']

17                  http://www.governoaberto.sp.gov.br/
18    http://www.camara.sp.gov.br/transparencia/dado...
23                  http://transparencia.tce.sp.gov.br/
25                   http://dados.prefeitura.sp.gov.br/
27              https://www.al.sp.gov.br/dados-abertos/
31             http://transparencia.campinas.sp.gov.br/
Name: URL, dtype: object

## excercicio-dados-sp (**frictionless**)
Converte, valida, carrega fonte de dados no banco SQL e cria vizualização simples


### 1. Instala pacotes Python externos

In [ ]:
pip install frictionless plotly plotly_express db-sqlite3

#### Imprime versões instaladas

In [ ]:
pip freeze | grep -e frictionless -e plotly -e db-sqlite3

db-sqlite3==0.0.1
frictionless==5.16.1
plotly==5.15.0
plotly-express==0.4.1


### 2. Converte arquivo XLSX em CSV

In [ ]:
import pandas as pd

In [ ]:
def convert_excel_to_csv(path, file, extension):
  content = pd.read_excel(f"{path}/{file}.{extension}")

  content.to_csv(f"{file}.csv", mode = "a", index = None, header = True)

  return f"{file}.csv"

In [ ]:
csv_file_path = convert_excel_to_csv("/content", "Estoque de Processos no Contencioso", "xls")
csv_file_path

### 3. Valida arquivo CSV

In [ ]:
from frictionless import validate

In [ ]:
# Descrevendo os dados

#description = describe(csv_file_path)
#pprint(description)

# Ajustar metadados caso necessário (ex.: field_missing_values, resource.schema.foreign_keys -> .yaml)

# Extraindo dados

#rows = extract(csv_file_path)
#pprint(rows)

#first_or_default = list(rows.values())[0]
#print(first_or_default)

In [ ]:
# Validando dados

report = validate(csv_file_path)
report

### 4. Cria banco de dados SQL local

In [ ]:
import sqlite3

In [ ]:
# Cria arquivo de banco de dados SQLite local -> PostgreSQL seria mais adequado
# para aplicativos de grande escala e necessidades de segurança avançadas.

db = sqlite3.connect("project.db")

### 5. Carrega arquivo CSV no banco de dados SQL

In [ ]:
from frictionless import formats, Resource

In [ ]:
# Cria recurso no formato 'Data Resource' a partir do arquivo .csv validado

resource = Resource(csv_file_path)
resource

In [ ]:
# Indexa/carrega dados do recurso 'Data Resource' na tabela 'idesp' do banco de dados

resource.index('sqlite:///project.db', name='idesp')
Resource('sqlite:///project.db', control=formats.sql.SqlControl(table='idesp')).extract()

### 6. Monta query p/ selecionar registros no banco de dados SQL

In [ ]:
query = """
  select * from idesp
"""

### 7. Cria data-frame e gráficos a patir dos registros selecionados

In [ ]:
df = pd.read_sql_query(query,db)
#df.head(5)

In [ ]:
import numpy as np
import plotly.express as px

# Plot 1: Pairplot using Plotly
fig = px.scatter_matrix(df)
fig.update_layout(title=f'Gráfico de Pares do Conjunto de Dados Desconhecido "{csv_file_path}"')
fig.show()

# Plot 2: Histogram of a numerical column
numerical_column = df.select_dtypes(include=[np.number]).columns[0]  # Select the first numerical column
fig = px.histogram(df, x=numerical_column, nbins=10, title=f'Histograma de {numerical_column}')
fig.show()

## web scraping

In [ ]:
pip install urllib3

In [ ]:
pip freeze | grep -e urllib3

urllib3==2.0.7


### Catálogo de Categorias

#### Inicialização (genérico)

In [ ]:
from bs4 import BeautifulSoup
import urllib3

# Disable SSL certificate verification
urllib3_pool_manager = urllib3.PoolManager(cert_reqs='CERT_NONE')

base_url = "http://catalogo.governoaberto.sp.gov.br"

#### Catálogo de Categorias - Recupera HTML (genérico)

In [ ]:
def get_page_html_by_url(categories_url):

  # Make a GET request to the website with SSL verification disabled
  http_response = urllib3_pool_manager.request("GET", categories_url)

  # Parse the HTML content using BeautifulSoup
  return BeautifulSoup(http_response.data, 'html.parser')

In [ ]:
categories_page_html = get_page_html_by_url(f"{base_url}{'/group'}")
categories_page_html

<!DOCTYPE html>

<!--[if IE 7]> <html lang="pt_BR" class="ie ie7"> <![endif]-->
<!--[if IE 8]> <html lang="pt_BR" class="ie ie8"> <![endif]-->
<!--[if IE 9]> <html lang="pt_BR" class="ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="pt_BR"> <!--<![endif]-->
<head>
<!--[if lte ie 8]><script type="text/javascript" src="/fanstatic/vendor/:version:2016-03-08T15:12:46/html5.min.js"></script><![endif]-->
<link href="/fanstatic/vendor/:version:2016-03-08T15:12:46/select2/select2.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/css/:version:2017-10-09T17:50:52/main.min.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/vendor/:version:2016-03-08T15:12:46/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<!--[if ie 7]><link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2016-03-08T15:12:46/font-awesome/css/font-awesome-ie7.min.css" /><![endif]-->
<link href="/fanstatic/rt_theme/:version:2017-10-09T14:43:07/:bundle:css/cus

#### Catálogo de Categorias - Recupera paginação (genérico)

In [ ]:
def get_pagination_urls(target_html):
  # Encontrando a div com a classe "pagination pagination-centered"
  pagination_element = target_html.find('div', class_='pagination pagination-centered')

  if pagination_element is None:
    return {}

  # Inicializando um dicionário vazio para armazenar os resultados
  result = {}

  # Encontrando todos os elementos <a> dentro da div
  for item in pagination_element.find_all('a'):
      page_number = item.text.strip()  # Obtendo o número da página
      page_url = item['href']  # Obtendo o URL da página
      if page_number.isdigit():  # Verificando se o número da página é um dígito
          result[page_number] = page_url

  # Obtendo a string desejada
  #page_url_prefix = result['1'].split('&page=')[0] + '&page='
  page_url_prefix = result['1'].split('=')[0] + '='

  # Preenchendo os valores faltantes no conjunto
  max_page_number = max(map(int, result.keys()), default=0)
  for i in range(1, max_page_number + 1):
      if str(i) not in result:
          result[str(i)] = f'{page_url_prefix}{i}'

  unique_urls = set()

  # Filtrando os valores duplicados
  filtered_dict = {}
  for page_number, page_url in result.items():
      if page_url not in unique_urls:
          filtered_dict[page_number] = page_url
          unique_urls.add(page_url)

  return dict(sorted(filtered_dict.items()))

In [ ]:
categories_page_urls = get_pagination_urls(categories_page_html)
categories_page_urls

{'1': '/group?page=1', '2': '/group?page=2'}

#### Catálogo de Categorias - Recupera título da página

In [ ]:
def get_categories_page_main_title(target_page):

  parent_element = target_page.find('form', id='group-search-form')

  target_element = parent_element.find_next('h2')

  return f"{target_element.text.strip()}"

In [ ]:
categories_page_main_title = get_categories_page_main_title(categories_page_html)
categories_page_main_title

'39 temas encontrados'

#### Catálogo de Categorias - Recupera itens das páginas do catálogo

In [ ]:
def get_categories_dict_from_category_page(target_html):

  category_items = target_html.find_all('li', class_='media-item')

  # Inicializar um dicionário vazio
  result = {}

  # Iterar sobre os itens e extrair os dados
  for item in category_items:
      title = item.find('h3', class_='media-heading').text
      link = item.find('a', class_='media-view')['href']
      result[title] = link

  return result

In [ ]:
def get_categories_dict_from_category_pages(page_urls_dict):

  categories_dict = {}

  # Percorre todas pagina de categoria
  for page_number, page_url in page_urls_dict.items():

      # Faz a requisicao HTTP
      resp = urllib3_pool_manager.request("GET", f"{base_url}{page_url}")

      # Parse the HTML content using BeautifulSoup
      target_page = BeautifulSoup(resp.data, 'html.parser')

      categories_dict.update(get_categories_dict_from_category_page(target_page))

  return categories_dict

In [ ]:
categories_dict = get_categories_dict_from_category_pages(categories_page_urls)
categories_dict

{'Administração pública': '/group/administracao-publica',
 'Agropecuária': '/group/agropecuaria',
 'Arquivos, bibliotecas e centros de informação': '/group/arquivos-bibliotecas-e-centros-de-informacao',
 'Arte e cultura': '/group/arte-e-cultura',
 'Ciência, tecnologia e inovação': '/group/ciencia-tecnologia-e-inovacao',
 'Comercialização de produtos': '/group/comercializacao-de-produtos',
 'Consultoria a empresas': '/group/consultoria-a-empresas',
 'Crédito, financiamento e serviços bancários': '/group/credito-financiamento-e-servicos-bancarios',
 'Defesa civil': '/group/defesa-civil',
 'Desenvolvimento Social': '/group/desenvolvimento-social',
 'Direitos e proteção do consumidor': '/group/direitos-e-protecao-do-consumidor',
 'Documentos pessoais': '/group/documentos-pessoais',
 'Educação': '/group/educacao',
 'Emergências': '/group/emergencias',
 'Empreendimentos imobiliários': '/group/empreendimentos-imobiliarios',
 'Energia': '/group/energia',
 'Esportes': '/group/esportes',
 'Habit

### Categoria específica

#### Categoria específica - Inicialização
Apenas 1a categoria por enquanto

In [ ]:
# Obtendo um iterador para os itens do dicionário
iterator = iter(categories_dict.items())

# Recuperando o primeiro item do dicionário
first_category_item = next(iterator)

#### Categoria específica - Recupera HTML

In [ ]:
category_page_html = get_page_html_by_url(f"{base_url}{first_category_item[1]}")
category_page_html

<!DOCTYPE html>

<!--[if IE 7]> <html lang="pt_BR" class="ie ie7"> <![endif]-->
<!--[if IE 8]> <html lang="pt_BR" class="ie ie8"> <![endif]-->
<!--[if IE 9]> <html lang="pt_BR" class="ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="pt_BR"> <!--<![endif]-->
<head>
<!--[if lte ie 8]><script type="text/javascript" src="/fanstatic/vendor/:version:2016-03-08T15:12:46/html5.min.js"></script><![endif]-->
<link href="/fanstatic/vendor/:version:2016-03-08T15:12:46/select2/select2.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/css/:version:2017-10-09T17:50:52/main.min.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/vendor/:version:2016-03-08T15:12:46/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<!--[if ie 7]><link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2016-03-08T15:12:46/font-awesome/css/font-awesome-ie7.min.css" /><![endif]-->
<link href="/fanstatic/rt_theme/:version:2017-10-09T14:43:07/:bundle:css/cus

#### Categoria específica - Recupera paginação de categoria específica

In [ ]:
category_page_urls = get_pagination_urls(category_page_html)
category_page_urls

{'1': '/group/administracao-publica?page=1',
 '2': '/group/administracao-publica?page=2',
 '3': '/group/administracao-publica?page=3',
 '4': '/group/administracao-publica?page=4',
 '5': '/group/administracao-publica?page=5',
 '6': '/group/administracao-publica?page=6'}

#### Categoria específica - Recupera título da página

In [ ]:
f"{base_url}{first_category_item[1]}"

'http://catalogo.governoaberto.sp.gov.br/group/administracao-publica'

In [ ]:
target_page = category_page_html

parent_element = target_page.find('form', id='group-datasets-search-form')

target_element = parent_element.find_next('h2')

print(f"{target_element.text.strip()}")

111 conjuntos de dados encontrados


In [ ]:
def get_category_page_main_title(target_page):

  parent_element = target_page.find('form', id='group-datasets-search-form')

  target_element = parent_element.find_next('h2')

  return f"{target_element.text.strip()}"

In [ ]:
category_page_main_title = get_category_page_main_title(category_page_html)
category_page_main_title

'111 conjuntos de dados encontrados'

### Dataset

#### Categoria específica - Recupera datasets das páginas da categoria

In [ ]:
def get_dataset_list_from_dataset_page(target_html):

  # Encontrar todas as entradas de conjunto de dados
  entries = target_html.find_all(class_='dataset-heading')

  # Inicializar a lista de datasets
  dataset_list = []

  # Iterar pelas entradas e extrair informações
  for entry in entries:
      dataset_name = entry.a.text
      dataset_url = entry.a['href']
      orgao = entry.find_next_sibling('h3').a.text
      descricao = entry.find_next_sibling('p').text
      formato = entry.find_next_sibling('div', class_="div-table stat")

      if formato is not None:
        formatos = [e['data-format'] for e in formato.find_all('a')]
      else:
          continue

      dataset_info = {
          'dataset_name': dataset_name,
          'url': dataset_url,
          'orgao': orgao,
          'desc': descricao,
          'formatos': formatos
      }

      dataset_list.append(dataset_info)

  # Retornar a lista de datasets resultante
  return dataset_list

In [ ]:
def get_dataset_list_from_dataset_pages(page_urls_dict):

  datasets_list = []

  # Percorre todas pagina de categoria
  for page_number, page_url in page_urls_dict.items():

      # Faz a requisicao HTTP
      resp = urllib3_pool_manager.request("GET", f"{base_url}{page_url}")

      # Parse the HTML content using BeautifulSoup
      target_page = BeautifulSoup(resp.data, 'html.parser')

      datasets_list.extend(get_dataset_list_from_dataset_page(target_page))

  return datasets_list

In [ ]:
dataset_list = get_dataset_list_from_dataset_pages(category_page_urls)
print(len(dataset_list))
dataset_list

In [ ]:
category_page_urls

#### Filtra datasets por formatos CSV, XLSX, XLS e JSON

In [ ]:
formatos_permitidos = ['csv', 'CSV']

In [ ]:
def filter_dataset_list_by_format(dataset_list, formatos_permitidos):

  # Filtrar os itens com 'formato' igual a 'csv', 'xlsx', 'xls' e 'json'
  #formatos_permitidos = ['csv', 'xlsx', 'xls', 'json']

  result = []

  for dataset_item in dataset_list:
    for formato in dataset_item['formatos']:
      if formato in formatos_permitidos:
        result.append(dataset_item)

  return result

In [ ]:
filtered_dataset_list = filter_dataset_list_by_format(dataset_list, formatos_permitidos)
print(len(filtered_dataset_list))
filtered_dataset_list[1]

#### Recupera detalhes de recursos de dataset específico

In [ ]:
def get_dataset_file_from_dataset_page(target_html):

  dataset_file_list = []

  for entry in target_html.find_all(class_='resource-item'):
    resource_title = entry.a["title"]
    format = entry.a.find_next("span")["data-format"]
    download_url = entry.div.find_next("ul").find_next("a").find_next("a")['href']

    last_dot_index = download_url.rfind('.')
    if download_url[last_dot_index + 1:] not in formatos_permitidos:
      continue

    if format not in formatos_permitidos:
      continue

    try:
      response = urllib3.PoolManager().request("GET", download_url)
      is_url_available = response.status == 200
    except urllib3.exceptions.HTTPError as e:
        is_url_available = False
    #is_url_available = urllib3_pool_manager.request("GET", download_url).status == 200

    dataset_file_info = {
      'resource_title': resource_title,
      'format': format,
      'download_url': download_url,
      'available': is_url_available
    }

    dataset_file_list.append(dataset_file_info)

  return dataset_file_list

In [ ]:
target_html = get_page_html_by_url(f"{base_url}{'/dataset/orcamento-publico-estadual'}")

dataset_resource_detail = get_dataset_file_from_dataset_page(target_html)

dataset_resource_detail

#### Realiza download de recurso de dataset em diretório local

In [ ]:
# URL do arquivo .csv que você deseja baixar
url = dataset_resource_detail[1]['download_url']

# Nome do arquivo .csv após o download
file_name = f"{dataset_resource_detail[1]['resource_title']}.{dataset_resource_detail[0]['format']}"

# Criando um objeto PoolManager do urllib3
#http = urllib3.PoolManager()

# Realizando a requisição GET para baixar o arquivo
response = urllib3_pool_manager.request('GET', url)

# Verificando se a requisição foi bem-sucedida
if response.status == 200:
    # Salvando o conteúdo do arquivo .csv no diretório local
    with open(file_name, 'wb') as f:
        f.write(response.data)
    print(f'O arquivo {file_name} foi baixado com sucesso!')
else:
    print('Falha ao baixar o arquivo. Verifique o link fornecido.')

O arquivo Despesa - Elementos.csv foi baixado com sucesso!


### Criacão de arquivo JSON

In [ ]:
import json

In [ ]:
#def create_json_data():
datasets = []

categories = []

#Recupera todas categorias disponiveis
categories_dict = get_categories_dict_from_category_pages(categories_page_urls)

for category_item in categories_dict.items():

  #Recupera 1a pagina da categoria
  category_page_html = get_page_html_by_url(f"{base_url}{category_item[1]}")

  #Recupera titulo da pagina da categoria
  category_page_main_title = get_category_page_main_title(category_page_html)

  #Recupera link das paginas subsequentes da categoria
  category_page_urls = get_pagination_urls(category_page_html)

  #Recupera conjuntos de dados da categoria (todas as paginas, apenas formatos suportados)
  dataset_complete_list = get_dataset_list_from_dataset_pages(category_page_urls)

  #Filtra apenas por formatos suportados
  supported_formats = ['csv', 'CSV']
  dataset_filtered_list = filter_dataset_list_by_format(dataset_complete_list, supported_formats)

  dataset_list = []
  for dataset_item in dataset_filtered_list:

      #Recupera detalhes dos recursos do conjunto de dados
      target_html = get_page_html_by_url(f"{base_url}{dataset_item['url']}")

      dataset_resource_detail = get_dataset_file_from_dataset_page(target_html)

      if len(dataset_resource_detail) > 0:
        dataset = {
            "name": dataset_item["dataset_name"],
            "entity": dataset_item["orgao"],
            "description": dataset_item["desc"],
            "resources": dataset_resource_detail
        }
        dataset_list.append(dataset)

  content_category = category_item[0]
  content_title = category_page_main_title
  content_datasets = dataset_list

  category = {
      "category": content_category,
      "title": f"{content_title}",
      "detail": f"Compatíveis com web-scrap (formato .csv): {len(content_datasets)}",
      "datasets": content_datasets
  }

  if len(content_datasets) > 0:
    categories.append(category)
    #print(categories)

with open("governo-aberto.json", "w", encoding="utf-8") as outfile:
      json.dump(categories, outfile, indent=4, ensure_ascii=False)

AttributeError: 'NoneType' object has no attribute 'find_next'

## Referências
 - SANTAREM SEGUNDO, Jose Eduardo. *Data Science para Humanas, Introdução a análise de dados, modelos e algoritmos de Machine Learning*. 26 de março de 2024. Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto, Ribeirão Preto, SP
 - https://www.youtube.com/dataprofessor (acesso em 03/04/2024)
 - https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb (acesso em 03/03/2024)
 - ChatGPT-4 (diversos prompts)
 - https://framework.frictionlessdata.io/ (acesso em 04/04/2024)
 - https://specs.frictionlessdata.io/data-resource/#language (acesso em 05/04/2024)
 - https://thivyapriyaa.medium.com/google-colab-series-setting-up-sqlite-de2aaaa02ec5 (acesso em 05/04/2024)
 - https://medium.com/@raftaarrashedin100/sqlite-connection-using-python-in-google-colab-6c8130fd672d (acesso em 05/04/2024)